In [8]:
import os
import pandas as pd

In [ ]:
folderPathPorData = '..\..\Bases\PorData\'

def read_csv_files_from_folders(folder_path):
    dataframes = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            print(f'Processing file: {root} {file}')
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
                key = os.path.splitext(os.path.relpath(file_path, folder_path))[0]
                dataframes[key] = df
    return dataframes


In [11]:
dataframes = read_csv_files_from_folders(folderPathPorData)

Processing file: ..\..\Bases\PorData\ outros
Processing file: ..\..\Bases\PorData\cultura 623-recintos-de-cinema.csv
Processing file: ..\..\Bases\PorData\cultura 631-sessoes-de-espetaculos-ao-vivo.csv
Processing file: ..\..\Bases\PorData\economia 294-depositos-de-clientes-nos-bancos-caixas-economicas-e.csv
Processing file: ..\..\Bases\PorData\economia 295-estabelecimentos-de-bancos-caixas-economicas-e-caixas.csv
Processing file: ..\..\Bases\PorData\economia Municipios_Poder-de-compra-per-capita.xlsx
Processing file: ..\..\Bases\PorData\educacao 375-estabelecimentos-de-ensino-superior-por-subsistema.csv
Processing file: ..\..\Bases\PorData\educacao 379-estabelecimentos-de-ensino-nao-superior-por-0.csv
Processing file: ..\..\Bases\PorData\emprego 439-desemprego-registado-nos-centros-de-emprego-por-grup.csv
Processing file: ..\..\Bases\PorData\emprego 445-populacao-empregada-por-conta-de-outrem-por-nivel-de.csv
Processing file: ..\..\Bases\PorData\emprego 447-populacao-empregada-por-sexo-

In [14]:
# liste os dataframes
for name, df in dataframes.items():
    print(f'{name}: {df.shape} - Columns: {list(df.columns)}')
    if 'Indicador' in df.columns:
        print(f"Indicador values in {name}: {df['Indicador'].unique()[:5]}")


DataFrame 0: (4689, 9) - Columns: ['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '05. Filtro 2', '06. Filtro 3', '07. Escala', '08. Símbolo', '09. Valor']
DataFrame 1: (4082, 9) - Columns: ['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '05. Filtro 2', '06. Filtro 3', '07. Escala', '08. Símbolo', '09. Valor']
DataFrame 2: (11128, 9) - Columns: ['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '05. Filtro 2', '06. Filtro 3', '07. Escala', '08. Símbolo', '09. Valor']
DataFrame 3: (3712, 9) - Columns: ['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '05. Filtro 2', '06. Filtro 3', '07. Escala', '08. Símbolo', '09. Valor']
DataFrame 4: (19471, 9) - Columns: ['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '05. Filtro 2', '06. Filtro 3', '07. Escala', '08. Símbolo', '09. Valor']
DataFrame 5: (155740, 9) - Columns: ['01. A